In [4]:
!pip install langchain langchain_community langchain_huggingface langchain_openai langgraph langchain_chroma langchain_anthropic langgraph-checkpoint-sqlite pypdf datasets ragas ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 48.6 MB/s e

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd "/content/drive/MyDrive/Documents/General reference/J/Job/Master Concept/"

/content/drive/MyDrive/Documents/General reference/J/Job/Master Concept


In [7]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [8]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./2022hkt_security_report.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

38


In [9]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

HKT Hong Kong Enterprise 
Cyber Security Readiness 
Index 2022 
 
April 2018
{'producer': '適用於 Microsoft 365 的 Microsoft® Word', 'creator': '適用於 Microsoft 365 的 Microsoft® Word', 'creationdate': '2022-11-18T12:34:46+08:00', 'author': 'HKCERT', 'moddate': '2022-11-18T12:45:00+08:00', 'title': 'HKT Hong Kong Enterprise Cyber Security Readiness Index 2022', 'source': './2022hkt_security_report.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}


In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=700)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [12]:
len(splits)

145

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What's the content of the document?"})

results

{'input': "What's the content of the document?",
 'context': [Document(id='36ed43c9-03a2-4e01-97a8-a5261bab41d7', metadata={'producer': '適用於 Microsoft 365 的 Microsoft® Word', 'creator': '適用於 Microsoft 365 的 Microsoft® Word', 'creationdate': '2022-11-18T12:34:46+08:00', 'author': 'HKCERT', 'moddate': '2022-11-18T12:45:00+08:00', 'title': 'HKT Hong Kong Enterprise Cyber Security Readiness Index 2022', 'source': './2022hkt_security_report.pdf', 'total_pages': 38, 'page': 1, 'page_label': '2'}, page_content='0 | P a g e \nTable of Content \n \n1. Introduction ................................................................................................ 1 \n1.1 Background ....................................................................................................................... 1 \n1.2 HKT Hong Kong Enterprise Cyber Security Readiness Index Survey ................................ 1 \n1.3 Thematic Survey of the Year ..............................................................

In [14]:
print(results["context"][0].page_content)

0 | P a g e 
Table of Content 
 
1. Introduction ................................................................................................ 1 
1.1 Background ....................................................................................................................... 1 
1.2 HKT Hong Kong Enterprise Cyber Security Readiness Index Survey ................................ 1 
1.3 Thematic Survey of the Year ............................................................................................ 2 
1.4 Structure of Report ........................................................................................................... 2 
2. Methodology .............................................................................................. 3 
2.1 Framework of the Index ................................................................................................... 3


In [15]:
print(results["context"][0].metadata)

{'producer': '適用於 Microsoft 365 的 Microsoft® Word', 'creator': '適用於 Microsoft 365 的 Microsoft® Word', 'creationdate': '2022-11-18T12:34:46+08:00', 'author': 'HKCERT', 'moddate': '2022-11-18T12:45:00+08:00', 'title': 'HKT Hong Kong Enterprise Cyber Security Readiness Index 2022', 'source': './2022hkt_security_report.pdf', 'total_pages': 38, 'page': 1, 'page_label': '2'}


In [16]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

## Set up agents for question generation

In [17]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.text_generation(
        prompt=prompt,
        max_new_tokens=1000
    )
    return response


call_llm(llm_client, "This is a test context")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


' for the `@mui/material` library.\n\n## Installation\n\n```sh\nnpm install @mui/material\n```\n\n## Usage\n\n```jsx\nimport React from \'react\';\nimport { Button } from \'@mui/material\';\n\nfunction App() {\n  return (\n    <div className="App">\n      <Button variant="contained" color="primary">\n        Hello World\n      </Button>\n    </div>\n  );\n}\n\nexport default App;\n```\n\n## Documentation\n\n- [Material-UI](https://material-ui.com/)\n- [Material Design](https://material.io/)'

In [18]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [19]:
from tqdm import tqdm

print(f"Generating {len(splits)} QA couples...")

outputs = []
for context in tqdm(splits):
    # Generate QA couple
    output_QA_couple = call_llm(llm_client, QA_generation_prompt.format(context=context.page_content))
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": context.metadata["source"],
            }
        )
    except:
        continue

Generating 145 QA couples...


100%|██████████| 145/145 [04:27<00:00,  1.84s/it]


In [20]:
display(pd.DataFrame(outputs).head(5))

,context,question,answer,source_doc
0,HKT Hong Kong Enterprise \nCyber Security Readiness \nIndex 2022 \n \nApril 2018,What is the name of the index in the context?\n,The name of the index in the context is the HKT Hong Kong Enterprise Cyber Security Readiness Index 2022.,./2022hkt_security_report.pdf
1,0 | P a g e \nTable of Content \n \n1. Introduction ................................................................................................ 1 \n1.1 Background ....................................................................................................................... 1 \n1.2 HKT Hong Kong Enterprise Cyber Security Readiness Index Survey ................................ 1 \n1.3 Thematic Survey of the Year ............................................................................................ 2 \n1.4 Structure of Report ........................................................................................................... 2 \n2. Methodology .............................................................................................. 3 \n2.1 Framework of the Index ................................................................................................... 3,What is the name of the survey mentioned in the context?\n,The name of the survey mentioned in the context is the HKT Hong Kong Enterprise Cyber Security Readiness Index Survey.,./2022hkt_security_report.pdf
2,1.2 HKT Hong Kong Enterprise Cyber Security Readiness Index Survey ................................ 1 \n1.3 Thematic Survey of the Year ............................................................................................ 2 \n1.4 Structure of Report ........................................................................................................... 2 \n2. Methodology .............................................................................................. 3 \n2.1 Framework of the Index ................................................................................................... 3 \n2.2 Sample Distribution .......................................................................................................... 4 \n2.3 Profile of Surveyed Enterprises ........................................................................................ 5 \n3. Survey Findings ........................................................................................... 7,What is the title of the report?\n,The title of the report is '1.2 HKT Hong Kong Enterprise Cyber Security Readiness Index Survey'.,./2022hkt_security_report.pdf
3,2. Methodology .............................................................................................. 3 \n2.1 Framework of the Index ................................................................................................... 3 \n2.2 Sample Distribution .......................................................................................................... 4 \n2.3 Profile of Surveyed Enterprises ........................................................................................ 5 \n3. Survey Findings ........................................................................................... 7 \n3.1 Cyber Security Environment ............................................................................................. 7 \n3.2 The Index ........................................................................................................................ 14 \n3.3 Thematic Survey of the Year: Managed Security Services (MSS) ................................... 22,What is the title of the thematic survey of the year in the index?\n,Managed Security Services (MSS),./2022hkt_security_report.pdf
4,2.3 Profile of Surveyed Enterprises ........................................................................................ 5 \n3. Survey Findings ........................................................................................... 7 \n3.1 Cyber Security Environment .....................

In [21]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to customers from the information technology sector.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the methodology of this report?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


 84%|████████▍ | 119/142 [1:06:55<13:10, 34.36s/it]

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

In [ ]:
# Save the final generated questions to save the generation process in the future
import os
if not os.path.exists("./data/datasets/"):
    os.mkdir("./data/datasets/")

with open('./data/datasets/generated_questions.json', 'w') as f:
    json.dump(generated_questions.to_dict(), f, indent=4)

In [ ]:
with open('./data/datasets/generated_questions.json', 'r') as f:
    generated_questions = json.load(f)
generated_questions = pd.DataFrame(generated_questions)
eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

# Build RAG System

## Indexing

In [ ]:
# Load
# The loading step is already done when we use a text splitter to split
# our pdf document into small chunks/documents with 1000 characters each

# Split
# Here we will further split each small document into snippets for retrieval
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

# Store
# Embed the snippets in vectors and store them in a vectorestore
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os

def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

## Retrieval and Generation

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM

def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 3,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

In [ ]:
for item in tqdm(eval_dataset):
    print(item['context'])
    break

# Evaluate RAG System

In [ ]:
from langchain_core.language_models import BaseChatModel


def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(question, llm, knowledge_index, reranker=reranker)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "context": example["context"],
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f, indent=4)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f, indent=4)

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200, 400, 600]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                splits,
                chunk_size=chunk_size,
                embedding_model_name=embeddings
            )

            print("Running RAG...")
            reranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0") if rerank else None
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

In [ ]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [ ]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(lambda x: int(x) if isinstance(x, str) else 1)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
settings = []
for setting in average_scores.index:
    chunk_size = setting.split('rag_chunk:')[1].split('_embeddings')[0]
    rerank = setting.split('rerank:')[1].split('_reader')[0]
    if rerank == 'True':
        settings.append('+'.join([chunk_size, 'reranker']))
    else:
        settings.append(chunk_size)
average_scores = pd.Series(average_scores.values, index=settings, name='eval_score_GPT4')
average_scores = average_scores * 100
average_scores.sort_values(inplace=True)
average_scores

In [ ]:
import plotly.express as px
fig = px.bar(
    average_scores,
    color=average_scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

In [ ]:
from ragas import SingleTurnSample
from ragas import evaluate
# from ragas.metrics import LLMContextPrecisionWithReference
from ragas.metrics import context_precision
from datasets import Dataset

# context_precision = LLMContextPrecisionWithReference()

# sample = SingleTurnSample(
#     user_input="Where is the Eiffel Tower located?",
#     reference="The Eiffel Tower is located in Paris.",
#     retrieved_contexts=["The Eiffel Tower is located in Paris."],
# )
data = {
"user_input":"Where is the Eiffel Tower located?",
"reference": "The Eiffel Tower is located in Paris.",
"retrieved_contexts":"The Eiffel Tower is located in Paris.",
}
dataset = Dataset.from_dict(data)
evaluate(dataset, [context_precision])
# await context_precision.single_turn_ascore(sample)

ArrowInvalid: Column 1 named reference expected length 34 but got length 37

In [ ]:
from ragas.metrics import LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas import evaluate

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]
results = evaluate(dataset=sample, metrics=metrics, llm=evaluator_llm,)

In [ ]:
# from ragas import SingleTurnSample
# from ragas.metrics import LLMContextPrecisionWithoutReference

# context_precision = LLMContextPrecisionWithoutReference(llm=llm)

sample = SingleTurnSample(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=["The Eiffel Tower is located in Paris."],
)
# await context_precision.single_turn_ascore(sample)

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper
from ragas import evaluate
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]
results = evaluate(dataset=sample, metrics=metrics, llm=evaluator_llm,)

AttributeError: 'SingleTurnSample' object has no attribute 'get_sample_type'